In [1]:
from pathlib import Path
from pyraphtory.context import PyRaphtory
from pyraphtory.vertex import Vertex
from pyraphtory.steps import Iterate, Step
!curl -o /tmp/lotr.csv https://raw.githubusercontent.com/Raphtory/Data/main/lotr.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 52206  100 52206    0     0   252k      0 --:--:-- --:--:-- --:--:--  252k


In [2]:
pr = PyRaphtory(spout_input=Path('/tmp/lotr.csv'), builder_script=Path('builder.py'),  builder_class='LotrGraphBuilder', mode='batch', logging=True).open()
rg = pr.graph()

b'WARNING: sun.reflect.Reflection.getCallerClass is not supported. This will impact performance.\n'
b"17:42:50.678 [io-compute-6] INFO  com.raphtory.spouts.FileSpout - Spout: Processing file 'lotr.csv' ...\n"
b'17:42:50.863 [io-compute-6] ERROR com.raphtory.internals.management.Prometheus$ - Failed to start Prometheus on port 9999\n'
b'17:42:50.879 [io-compute-6] INFO  com.raphtory.internals.management.Prometheus$ - Prometheus started on port /0:0:0:0:0:0:0:0:53608\n'
b'17:42:52.102 [spawner-akka.actor.default-dispatcher-3] INFO  akka.event.slf4j.Slf4jLogger - Slf4jLogger started\n'
b"17:42:52.494 [io-compute-6] INFO  com.raphtory.internals.management.PartitionsManager$ - Creating '1' Partition Managers for raphtory_753033557.\n"
b'17:42:52.740 [io-compute-6] INFO  com.raphtory.internals.management.Py4JServer - Starting PythonGatewayServer...\n'
b'17:42:52.743 [io-compute-6] INFO  com.raphtory.internals.management.Py4JServer - Started PythonGatewayServer on port 53609 host: localhost/1

In [3]:
local_sink = pr.local_sink()

cols = ["inDegree", "outDegree", "degree","triangleCount","prlabel","cclabel","twoHopPaths"]

tracker = rg.at(32000)\
                .past()\
                .transform(pr.page_rank())\
                .transform(pr.connected_components())\
                .transform(pr.degree())\
                .transform(pr.two_hops_path(set([])))\
                .transform(pr.local_triangle_count())\
                .select(cols) \
                .write_to(local_sink)\
                .wait_for_job()


pr.results_to_dataframe(local_sink.results(),cols)



AttributeError: 'TemporalGraph' object has no attribute 'range'

In [ ]:
class PGStep(Step):
    def eval(self, v: Vertex):
        initLabel = 1.0
        v["prlabel"] = initLabel
        out_degree = v.out_degree()
        if out_degree > 0:
            msg = initLabel / out_degree
            v.message_outgoing_neighbours(msg)

class PGIterate(Iterate):
    def __init__(self, iterations: int, execute_messaged_only: bool, damping_factor: float = 0.85):
        super().__init__(iterations, execute_messaged_only)
        self.damping_factor = damping_factor

    def eval(self, v: Vertex):
        current_label = v["prlabel"]
        queue = v.message_queue()
        summed_queue = sum(queue)
        new_label = (1 - self.damping_factor) + self.damping_factor * summed_queue
        v["prlabel"] = new_label

        out_degree = v.out_degree()

        if out_degree > 0:
            v.message_outgoing_neighbours(new_label / out_degree)

        if abs(new_label - current_label) < 0.00001:
            v.vote_to_halt()

In [ ]:
from pyraphtory.builder import *

class LotrGraphBuilder(BaseBuilder):
    def __init__(self):
        super(LotrGraphBuilder, self).__init__()

    def parse_tuple(self, line: str):
        src_node, target_node, timestamp, *_ = line.split(",")

        src_id = self.assign_id(src_node)
        tar_id = self.assign_id(target_node)

        self.add_vertex(int(timestamp), src_id, [ImmutableProperty("name", src_node)], "Character")
        self.add_vertex(int(timestamp), tar_id, [ImmutableProperty("name", target_node)], "Character")
        self.add_edge(int(timestamp), src_id, tar_id, [], "Character Co-occurence")